In [1]:
import requests
import pandas as pd
import time
import json
from typing import Dict, List, Any

# CONFIG
INPUT_FILE = "eissn.txt"
OUTPUT_FILE = "journal_metadata.xlsx"
REQUEST_DELAY = 1  # seconds between API requests

# Read E-ISSNs
with open(INPUT_FILE, "r") as f:
    eissn_list = [line.strip() for line in f if line.strip()]

# Get ROR ID by publisher name
def get_ror_id(publisher_name):
    try:
        res = requests.get(f"https://api.ror.org/organizations?query={publisher_name}")
        if res.status_code == 200:
            items = res.json().get("items", [])
            if items:
                org = items[0]
                return org.get("id", ""), org.get("name", "")
    except Exception as e:
        print(f"❌ ROR error for '{publisher_name}': {e}")
    return "", ""

# Try DOAJ first
def try_doaj(issn):
    try:
        res = requests.get(f"https://doaj.org/api/v4/search/journals/issn:{issn}?page=1&pageSize=1")
        if res.status_code == 200:
            data = res.json()
            if data["total"] > 0:
                journal = data["results"][0]
                bib = journal["bibjson"]

                # Publisher information
                publisher_name = bib.get("publisher", {}).get("name", "")
                publisher_country = bib.get("publisher", {}).get("country", "")
                journal_url = bib.get("ref", {}).get("journal", "")

                # License extraction
                license_info = ""
                license_url = ""
                if bib.get("license") and len(bib["license"]) > 0:
                    license_info = bib["license"][0].get("type", "")
                    license_url = bib["license"][0].get("url", "")

                # APC extraction
                apc_info = "No"
                apc_amount = ""
                apc_currency = ""
                if bib.get("apc", {}).get("has_apc", False):
                    apc_info = "Yes"
                    # Get the APC amount if available
                    if bib.get("apc", {}).get("max") and len(bib["apc"]["max"]) > 0:
                        apc_amount = str(bib["apc"]["max"][0].get("price", ""))
                        apc_currency = bib["apc"]["max"][0].get("currency", "")

                # Print ISSN
                p_issn = bib.get("pissn", "")

                # Subject categories
                subjects = []
                for subj in bib.get("subject", []):
                    subjects.append(f"{subj.get('term', '')} ({subj.get('scheme', '')})")

                # Editorial info
                review_process = ", ".join(bib.get("editorial", {}).get("review_process", []))

                # Publication timeline
                publication_time = bib.get("publication_time_weeks", "")

                # OA details
                oa_start_year = bib.get("oa_start", "")
                boai_compliant = "Yes" if bib.get("boai", False) else "No"

                # Keywords
                keywords = ", ".join(bib.get("keywords", []))

                # Academic identifiers
                has_doi = "No"
                if bib.get("pid_scheme", {}).get("has_pid_scheme", False) and "DOI" in bib.get("pid_scheme", {}).get("scheme", []):
                    has_doi = "Yes"

                ror_id, ror_name = get_ror_id(publisher_name)

                return {
                    "Source": "DOAJ",
                    "E-ISSN": issn,
                    "P-ISSN": p_issn,
                    "Title": bib.get("title", ""),
                    "Publisher": publisher_name,
                    "Publisher Country": publisher_country,
                    "ROR Name": ror_name,
                    "ROR ID": ror_id,
                    "URL": journal_url,
                    "License": license_info,
                    "License URL": license_url,
                    "Language": ", ".join(bib.get("language", [])),
                    "APC": apc_info,
                    "APC Amount": apc_amount,
                    "APC Currency": apc_currency,
                    "Subjects": "; ".join(subjects),
                    "Keywords": keywords,
                    "Review Process": review_process,
                    "Publication Time (weeks)": publication_time,
                    "OA Start Year": oa_start_year,
                    "BOAI Compliant": boai_compliant,
                    "Uses DOI": has_doi,
                    "Works Count": "",
                    "Cited By Count": "",
                    "h-index": "",
                    "Topics": ""
                }
    except Exception as e:
        print(f"❌ DOAJ error for {issn}: {e}")
    return {}

# Try OpenAlex next
def try_openalex(issn):
    try:
        res = requests.get(f"https://api.openalex.org/sources/issn:{issn}")
        if res.status_code == 200:
            data = res.json()

            # Extract topics
            topics = []
            if "topics" in data:
                for i, topic in enumerate(data["topics"]):
                    if i < 5:  # Limit to top 5 topics
                        topics.append(topic.get("display_name", ""))

            # Extract publisher/organization info
            publisher_name = data.get("host_organization_name", "")
            publisher_country = data.get("country_code", "")

            # Get ROR ID for the publisher
            ror_id, ror_name = "", ""
            if publisher_name:
                ror_id, ror_name = get_ror_id(publisher_name)

            # APC information
            apc_info = "No"
            apc_amount = ""
            apc_currency = ""

            if data.get("apc_prices") and len(data["apc_prices"]) > 0:
                apc_info = "Yes"
                apc_amount = str(data["apc_prices"][0].get("price", ""))
                apc_currency = data["apc_prices"][0].get("currency", "")

            # Get P-ISSN
            p_issn = ""
            if "issn" in data and isinstance(data["issn"], list) and len(data["issn"]) > 0:
                for issn_val in data["issn"]:
                    if issn_val != issn:  # If it's not the E-ISSN we're looking for
                        p_issn = issn_val
                        break

            # Determine if using DOI
            uses_doi = "Yes" if data.get("works_count", 0) > 0 else "Unknown"

            return {
                "Source": "OpenAlex",
                "E-ISSN": issn,
                "P-ISSN": p_issn,
                "Title": data.get("display_name", ""),
                "Publisher": publisher_name,
                "Publisher Country": publisher_country,
                "ROR Name": ror_name,
                "ROR ID": ror_id,
                "URL": data.get("homepage_url", ""),
                "License": "",
                "License URL": "",
                "Language": "",
                "APC": apc_info,
                "APC Amount": apc_amount,
                "APC Currency": apc_currency,
                "Subjects": "",
                "Keywords": "",
                "Review Process": "",
                "Publication Time (weeks)": "",
                "OA Start Year": "",
                "BOAI Compliant": "Yes" if data.get("is_oa", False) else "No",
                "Uses DOI": uses_doi,
                "Works Count": str(data.get("works_count", "")),
                "Cited By Count": str(data.get("cited_by_count", "")),
                "h-index": str(data.get("summary_stats", {}).get("h_index", "")),
                "Topics": "; ".join(topics)
            }
    except Exception as e:
        print(f"❌ OpenAlex error for {issn}: {e}")
    return {}

# Try Crossref if DOAJ and OpenAlex fail
def try_crossref(issn):
    try:
        res = requests.get(f"https://api.crossref.org/journals/{issn}")
        if res.status_code == 200:
            msg = res.json()["message"]
            publisher = msg.get("publisher", "")

            # Extract DOI prefix if available
            doi_prefix = ""
            if "doi-prefixes" in msg and msg["doi-prefixes"]:
                doi_prefix = msg["doi-prefixes"][0].get("prefix", "")

            # Extract P-ISSN from ISSN array if available
            p_issn = ""
            if "ISSN" in msg and isinstance(msg["ISSN"], list):
                # Filter out the E-ISSN
                for issn_item in msg.get("ISSN", []):
                    if issn_item != issn:  # If it's not the E-ISSN we queried
                        p_issn = issn_item
                        break

            # URL extraction
            journal_url = msg.get("URL", "")

            # Try to get URL from links if direct URL is not available
            if not journal_url and "link" in msg:
                for link in msg["link"]:
                    if link.get("content-type", "").startswith("text/html"):
                        journal_url = link.get("URL", "")
                        break

            ror_id, ror_name = get_ror_id(publisher)

            return {
                "Source": "Crossref",
                "E-ISSN": issn,
                "P-ISSN": p_issn,
                "Title": msg.get("title", ""),
                "Publisher": publisher,
                "Publisher Country": "",
                "ROR Name": ror_name,
                "ROR ID": ror_id,
                "URL": journal_url,
                "License": "",
                "License URL": "",
                "Language": "",
                "APC": "",
                "APC Amount": "",
                "APC Currency": "",
                "Subjects": "",
                "Keywords": "",
                "Review Process": "",
                "Publication Time (weeks)": "",
                "OA Start Year": "",
                "BOAI Compliant": "",
                "Uses DOI": "Yes" if doi_prefix else "",
                "DOI Prefix": doi_prefix,
                "Works Count": "",
                "Cited By Count": "",
                "h-index": "",
                "Topics": ""
            }
    except Exception as e:
        print(f"❌ Crossref error for {issn}: {e}")
    return {}

# Get Crossref DOI information regardless of source
def get_crossref_doi_info(issn):
    try:
        res = requests.get(f"https://api.crossref.org/journals/{issn}")
        if res.status_code == 200:
            msg = res.json()["message"]
            # Extract DOI prefix if available
            doi_info = {
                "DOI Prefix": "",
                "Crossref Member ID": ""
            }

            if "doi-prefixes" in msg and msg["doi-prefixes"]:
                doi_info["DOI Prefix"] = msg["doi-prefixes"][0].get("prefix", "")

            if "member" in msg:
                doi_info["Crossref Member ID"] = str(msg["member"])

            return doi_info
    except Exception as e:
        print(f"⚠️ Crossref DOI lookup failed for {issn}: {e}")
    return {}

# Combined lookup with cross-matching from all three sources
def get_journal_data(issn):
    print(f"  Processing {issn}...")

    # Initialize an empty result dictionary with all fields
    result = {
        "Source": "",
        "E-ISSN": issn,
        "P-ISSN": "",
        "Title": "",
        "Publisher": "",
        "Publisher Country": "",
        "ROR Name": "",
        "ROR ID": "",
        "URL": "",
        "License": "",
        "License URL": "",
        "Language": "",
        "APC": "",
        "APC Amount": "",
        "APC Currency": "",
        "Subjects": "",
        "Keywords": "",
        "Review Process": "",
        "Publication Time (weeks)": "",
        "OA Start Year": "",
        "BOAI Compliant": "",
        "Uses DOI": "",
        "DOI Prefix": "",
        "Crossref Member ID": "",
        "Works Count": "",
        "Cited By Count": "",
        "h-index": "",
        "Topics": ""
    }

    # Track which sources were successfully queried
    sources = []

    # Try DOAJ
    print("    🔍 Querying DOAJ...")
    doaj_data = try_doaj(issn)
    if doaj_data:
        sources.append("DOAJ")
        print("    ✓ DOAJ data found")
    else:
        print("    ✗ No DOAJ data")

    # Try OpenAlex
    print("    🔍 Querying OpenAlex...")
    openalex_data = try_openalex(issn)
    if openalex_data:
        sources.append("OpenAlex")
        print("    ✓ OpenAlex data found")
    else:
        print("    ✗ No OpenAlex data")

    # Try Crossref
    print("    🔍 Querying Crossref...")
    crossref_data = try_crossref(issn)
    if crossref_data:
        sources.append("Crossref")
        print("    ✓ Crossref data found")
    else:
        print("    ✗ No Crossref data")

    # Get DOI info from Crossref regardless of source
    doi_info = get_crossref_doi_info(issn)
    if doi_info and doi_info.get("DOI Prefix"):
        sources.append("Crossref-DOI")
        print("    ✓ Crossref DOI info found")

    if not sources:
        print("    ❌ Journal not found in any source")
        result["Source"] = "Not found"
        return result

    # Merge data from all sources
    print("    🔄 Merging data from all sources...")

    # Fill in data from all sources, prioritizing in this order:
    all_data_sources = [
        (doaj_data, "DOAJ"),
        (openalex_data, "OpenAlex"),
        (crossref_data, "Crossref"),
    ]

    filled_fields = set()
    field_sources = {}

    # First pass: fill in data from each source if field is empty
    for data, source_name in all_data_sources:
        if not data:
            continue

        for key, value in data.items():
            if key in result and not result[key] and value:
                result[key] = value
                filled_fields.add(key)
                field_sources[key] = source_name

    # Special handling for DOI info
    if doi_info:
        for key, value in doi_info.items():
            if key in result and not result[key] and value:
                result[key] = value
                filled_fields.add(key)
                field_sources[key] = "Crossref-DOI"

    # Add special fields that are better in certain sources
    if openalex_data:
        # OpenAlex has the best bibliometric data
        for key in ["Works Count", "Cited By Count", "h-index", "Topics"]:
            if key in openalex_data and openalex_data[key]:
                result[key] = openalex_data[key]
                filled_fields.add(key)
                field_sources[key] = "OpenAlex"

    # Set the source field to show data sources used
    result["Source"] = " + ".join(sources)

    # Log which fields came from where
    source_summary = {}
    for key, source in field_sources.items():
        if source not in source_summary:
            source_summary[source] = []
        source_summary[source].append(key)

    print("    📊 Data sources used:")
    for source, fields in source_summary.items():
        print(f"      - {source}: {len(fields)} fields")

    return result

# Run lookup
results = []
print("🚀 Starting lookup...\n")
for i, issn in enumerate(eissn_list, 1):
    print(f"[{i}/{len(eissn_list)}] Processing {issn}")
    data = get_journal_data(issn)
    results.append(data)
    time.sleep(REQUEST_DELAY)

# Save to Excel (original)
df = pd.DataFrame(results)
# df.to_excel(OUTPUT_FILE, index=False)  # Comment out or remove this line
json_output_file = "journal_metadata.json"

# Define the field types explicitly and clean up field names
typed_results: List[Dict[str, Any]] = []
for item in results:
    # Create a new dictionary with clean keys
    cleaned_item = {}
    for k, v in item.items():
        # Replace spaces and dashes with underscores in keys
        clean_key = k.replace(" ", "_").replace("-", "_")
        # Ensure all values are strings
        # For empty values, use empty string instead of "None"
        cleaned_value = "" if v is None else str(v).strip()
        cleaned_item[clean_key] = cleaned_value
    typed_results.append(cleaned_item)

# Add metadata about the export
final_data = {
    "metadata": {
        "generated_at": time.strftime("%Y-%m-%d %H:%M:%S"),
        "count": len(typed_results),
        "source": "Journal Data Fetcher",
        "version": "1.0"
    },
    "journals": typed_results
}

with open(json_output_file, 'w', encoding='utf-8') as f:
    json.dump(final_data, f, indent=2, ensure_ascii=False)

print(f"\n✅ Done! {len(typed_results)} journals saved to: {json_output_file}")

df.head()


🚀 Starting lookup...

[1/19] Processing 2715-7415
  Processing 2715-7415...
    🔍 Querying DOAJ...
    ✓ DOAJ data found
    🔍 Querying OpenAlex...
    ✗ No OpenAlex data
    🔍 Querying Crossref...
    ✓ Crossref data found
    🔄 Merging data from all sources...
    📊 Data sources used:
      - DOAJ: 21 fields
[2/19] Processing 2620-3081
  Processing 2620-3081...
    🔍 Querying DOAJ...
    ✓ DOAJ data found
    🔍 Querying OpenAlex...
    ✓ OpenAlex data found
    🔍 Querying Crossref...
    ✓ Crossref data found
    🔄 Merging data from all sources...
    📊 Data sources used:
      - DOAJ: 21 fields
      - OpenAlex: 4 fields
[3/19] Processing 2406-8780
  Processing 2406-8780...
    🔍 Querying DOAJ...
    ✓ DOAJ data found
    🔍 Querying OpenAlex...
    ✓ OpenAlex data found
    🔍 Querying Crossref...
    ✓ Crossref data found
    🔄 Merging data from all sources...
    📊 Data sources used:
      - DOAJ: 21 fields
      - OpenAlex: 4 fields
[4/19] Processing 1858-005X
  Processing 1858-00

,Source,E-ISSN,P-ISSN,Title,Publisher,Publisher Country,ROR Name,ROR ID,URL,License,...,Publication Time (weeks),OA Start Year,BOAI Compliant,Uses DOI,DOI Prefix,Crossref Member ID,Works Count,Cited By Count,h-index,Topics
0,DOAJ + Crossref,2715-7415,2088-205X,Tekno - Pedagogi,Magister Teknologi Pendidikan Universitas Jambi,ID,Jambi University,https://ror.org/00g1w3j30,https://online-journal.unja.ac.id/pedagogi/index,CC BY,...,16,2011,Yes,Yes,,,,,,
1,DOAJ + OpenAlex + Crossref,2620-3081,1411-2744,Jurnal Teknologi Pendidikan,Universitas Negeri Jakarta,ID,State University of Jakarta,https://ror.org/01hgg7b81,https://journal.unj.ac.id/unj/index.php/jtp,CC BY-SA,...,12,2016,Yes,Yes,,,154,1869,16,Educational Methods and Media Use; Educational...
2,DOAJ + OpenAlex + Crossref,2406-8780,2406-8780,Jurnal Inovasi dan Teknologi Pembelajaran,Universitas Negeri Malang,ID,State University of Malang,https://ror.org/00ypgyy34,http://journal2.um.ac.id/index.php/jinotep/index,CC BY-SA,...,12,2014,Yes,Yes,,,159,1531,18,Educational Methods and Media Use; Educational...
3,DOAJ + OpenAlex + Crossref,1858-005X,1858-005X,Edusaintek,STKIP PGRI Situbondo,ID,Sekolah Tinggi Keguruan dan Ilmu Pendidikan PG...,https://ror.org/01gedh423,https://journalstkippgrisitubondo.ac.id/index....,CC BY-SA,...,6,2021,Yes,Yes,,,96,234,10,Educational Methods and Media Use; Education a...
4,DOAJ + OpenAlex + Crossref,2407-0963,2407-0963,Jurnal Inovasi Teknologi Pendidikan,Universitas Negeri Yogyakarta,ID,Yogyakarta State University,https://ror.org/05fryw881,https://journal.uny.ac.id/index.php/jitp/index,CC BY-SA,...,6,2014,Yes,Yes,,,186,2907,29,Educational Methods and Media Use; STEM Educat...
